# BW \#67 Eletric Cars
One way to increase the number of EVs in the United States would be to import the inexpensive Chinese-manufactured vehicles that are so popular elsewhere. But many experts and governments believe that the Chinese government is subsidizing these manufacturers, allowing them to price the cars below what would be the normal rate. The Biden administration worries, not unreasonably, that a flood of cheap Chinese EVs would be disastrous for the American auto industry. As a result, they recently announced massive import taxes on EVs manufactured in China. 
I couldn't find any good data on the number of Chinese vehicles, so I decided to stick with the IEA (International Energy Association, https://iea.org) data on the share of various kinds of cars manufactured and sold in various countries. In this way, I hope that we can get a better sense of how popular EVs are, and where.

## Data and six questions
This week's data comes from the IEA's latest "Global EV outlook" (https://www.iea.org/reports/global-ev-outlook-2024). The data is downloadable from their "data explorer" page:

https://www.iea.org/data-and-statistics/data-tools/global-ev-data-explorer

You can download the data, or parts of it, by going to the above URL and selecting "ev sales," "cars," and "world" on the pull-down menus. Then click on "download data," and the CSV file should be downloaded to you.

## Challenges
The learning goals include filtering, grouping, multi-indexes, pivoting, piping, and also styling Pandas data frames.
- Read the EV data into a data frame. Remove rows from the "world" region. Keep only those rows with "EV sales" and "EV stock" parameters.
- Create a data frame showing the number of cars sold each year, in each country, with a BEV ("battery electric vehicle") powertrain.


In [1]:
import pandas as pd

In [22]:
%%timeit
filename = "C:\\Users\\npigeon\\Git\\BW #67 Electric cars\\IEA-EV-dataEV salesHistoricalCars.csv"
df = pd.read_csv(filename,
                 ).query("(region != 'World') & (parameter == 'EV sales' | parameter == 'EV stock')") # supprimer les row where region = World, et garder les row où parameter = EV sales ou EV stock
df

9.63 ms ± 220 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


L'utilisation de .loc avec lambda permet de chaîner des opérations de filtrage de manière élégante et lisible.
- lambda df_: Une fonction anonyme qui prend le DataFrame comme argument.
- df_['parameter'].isin(['EV sales', 'EV stock']): Crée une série de booléens où chaque valeur est True si la colonne parameter contient 'EV sales' ou 'EV stock'.
- .loc[...]: Utilise cette série de booléens pour filtrer le DataFrame, ne gardant que les lignes où la condition est True.

In [33]:
df = (
    pd
    .read_csv(filename)    
    .loc[lambda df_: df_['parameter'].isin(['EV sales', 'EV stock'])]
    .loc[lambda df_: df_['region'] != 'World']
)
df

,region,category,parameter,mode,powertrain,year,unit,value
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
5,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0
...,...,...,...,...,...,...,...,...
3638,USA,Historical,EV stock,Cars,PHEV,2023,Vehicles,1300000.0
3640,USA,Historical,EV stock,Cars,FCEV,2023,Vehicles,18000.0
3641,USA,Historical,EV sales,Cars,FCEV,2023,Vehicles,3000.0
3642,USA,Historical,EV stock,Cars,BEV,2023,Vehicles,3500000.0


#### Create a data frame showing the number of cars sold each year, in each country, with a BEV ("battery electric vehicle") powertrain.

In [37]:
(df
    .loc[lambda df_:df_['parameter'] == 'EV sales']
    .loc[lambda df_:df_['powertrain'] == 'BEV']
    .groupby(['region', 'year'])['value'].sum().reset_index()
)
df

,region,category,parameter,mode,powertrain,year,unit,value
2,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
5,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0
...,...,...,...,...,...,...,...,...
3638,USA,Historical,EV stock,Cars,PHEV,2023,Vehicles,1300000.0
3640,USA,Historical,EV stock,Cars,FCEV,2023,Vehicles,18000.0
3641,USA,Historical,EV sales,Cars,FCEV,2023,Vehicles,3000.0
3642,USA,Historical,EV stock,Cars,BEV,2023,Vehicles,3500000.0


In [36]:
(
    df
    .loc[lambda df_: df_['parameter'] == 'EV sales']
    .loc[lambda df_: df_['powertrain'] == 'BEV']
    .pivot_table(index='year',
                columns='region',
                values='value')   
)

region,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Costa Rica,...,Slovakia,Slovenia,South Africa,Spain,Sweden,Switzerland,Turkiye,USA,United Arab Emirates,United Kingdom
year,,,,,,,,,,,,,,,,,,,,,
2010,NaN,NaN,48.0,10.0,NaN,NaN,NaN,1100.0,NaN,NaN,...,NaN,NaN,NaN,76.0,4.0,NaN,NaN,1200.0,NaN,240.0
2011,49.0,NaN,280.0,8.0,NaN,220.0,6.0,4800.0,NaN,NaN,...,NaN,NaN,NaN,390.0,180.0,400.0,NaN,9800.0,NaN,1200.0
2012,170.0,NaN,570.0,22.0,NaN,620.0,5.0,9600.0,NaN,NaN,...,NaN,NaN,NaN,430.0,270.0,340.0,92.0,15000.0,NaN,1600.0
2013,190.0,650.0,500.0,39.0,NaN,1600.0,5.0,15000.0,NaN,NaN,...,NaN,NaN,34.0,810.0,430.0,1200.0,NaN,48000.0,NaN,2600.0
2014,370.0,1700.0,1300.0,61.0,NaN,2800.0,3.0,49000.0,NaN,NaN,...,NaN,NaN,14.0,1400.0,1200.0,2700.0,46.0,63000.0,NaN,6700.0
2015,760.0,1700.0,1400.0,61.0,21.0,4400.0,10.0,150000.0,39.0,4.0,...,52.0,NaN,120.0,1300.0,3000.0,3300.0,120.0,71000.0,NaN,10000.0
2016,670.0,3800.0,2100.0,130.0,5.0,5200.0,18.0,260000.0,28.0,12.0,...,59.0,140.0,100.0,2000.0,2900.0,3300.0,44.0,87000.0,NaN,10000.0
2017,1200.0,5400.0,2700.0,140.0,68.0,8700.0,120.0,470000.0,92.0,25.0,...,210.0,290.0,68.0,3900.0,4400.0,4800.0,77.0,100000.0,NaN,14000.0
2018,1800.0,6800.0,3600.0,180.0,190.0,23000.0,100.0,820000.0,250.0,200.0,...,290.0,470.0,66.0,6000.0,7100.0,5100.0,190.0,240000.0,59.0,16000.0


The result is a rejiggering of our data, letting us see just how many EVs were sold in each country over the last number of years.

In many cases, the differences are staggering — such as 4 (yes, four) vehicles in Sweden in 2010, and 110,000 in 2013. Or 1,100 vehicles in China in 2010, versus 5,400,000 in 2013.